In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf


In [20]:
data=pd.read_csv('metro data.csv')
data2=pd.read_csv('Eng_hassan.csv')

In [65]:
data2=data2[['x1','y1']]
out=[14,14,14,14,15,14,13,13,13]

In [66]:
data2[0:4]

,x1,y1
0,-1.19751,-30.2887
1,-1.19507,-29.4830
2,-1.19263,-28.3844
3,-1.19019,-27.8137


In [68]:
#492.0 horizon 1
def custom_ts_multi_data_prep2(dataset,start,end,window,horizon):
  X=[]
  Y=[]
  start=start+window
  if end is None:
    end=len(dataset)

  for i in range(start,end+1,492):

    indices=range(i-window,i)
    ss=i-window
    ee=i

    X.append(dataset[ss:ee])

  return np.array(X)


In [69]:
x_t2=custom_ts_multi_data_prep2(data2,0,None,492,1)

In [72]:
x_t2.shape

X_scaler=MinMaxScaler()
X_data=X_scaler.fit_transform(data2[['x1','y1']])


In [74]:
y_t2.shape

(9,)

In [10]:
for i in data.select_dtypes('object').columns:
  le=LabelEncoder().fit(data[i])
  data[i]=le.transform(data[i])

X_scaler=MinMaxScaler()
Y_scaler=MinMaxScaler()
X_data=X_scaler.fit_transform(data[['holiday','temp','rain_1h','snow_1h','clouds_all','weather_main','weather_description']])
Y_data=Y_scaler.fit_transform(data[['traffic_volume']])

In [11]:
def custom_ts_multi_data_prep(dataset,target,start,end,window,horizon):
  X=[]
  Y=[]
  start=start+window
  if end is None:
    end=len(dataset)-horizon

  for i in range(start,end):
    indices=range(i-window,i)
    X.append(dataset[indices])
    indicey=range(i+1,i+1+horizon)
    Y.append(target[indicey])
  return np.array(X),np.array(Y)

In [12]:
hist_window=48
horizon=1
Train_split=5
x_train,y_train=custom_ts_multi_data_prep(X_data,Y_data,0,Train_split,hist_window,horizon)
x_test,y_test=custom_ts_multi_data_prep(X_data,Y_data,Train_split,None,hist_window,horizon)

In [13]:
print(x_train[0].shape)

(48, 7)


In [14]:
batch_size=256
buffer_size=150
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()
test_data=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_data=test_data.batch(batch_size).repeat()

In [15]:
x_train.shape[-2:]

(48, 7)

In [16]:
model=tf.keras.models.Sequential([
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True),input_shape=x_train.shape[-2:]),
      tf.keras.layers.Dense(20,activation='tanh'),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
      tf.keras.layers.Dense(20,activation='tanh'),
      tf.keras.layers.Dense(20,activation='tanh'),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Dense(units=horizon),


])
model.compile(optimizer='adam',loss='mse')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 48, 400)          332800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 48, 20)            8020      
                                                                 
 bidirectional_1 (Bidirectio  (None, 300)              205200    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 20)                6020      
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dropout (Dropout)           (None, 20)                0

In [18]:
model.fit(train_data,epochs=100,steps_per_epoch=100)

Epoch 1/100
100/100 [==============================] - 20s 87ms/step - loss: 0.1115
Epoch 2/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0877
Epoch 3/100
100/100 [==============================] - 9s 86ms/step - loss: 0.0833
Epoch 4/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0804
Epoch 5/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0798
Epoch 6/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0774
Epoch 7/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0780
Epoch 8/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0785
Epoch 9/100
100/100 [==============================] - 9s 86ms/step - loss: 0.0759
Epoch 10/100
100/100 [==============================] - 9s 86ms/step - loss: 0.0762
Epoch 11/100
100/100 [==============================] - 9s 88ms/step - loss: 0.0753
Epoch 12/100
100/100 [==============================] - 9s 85ms/step - loss: 0.0756
